In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import xarray as xr
import scipy
from glob import glob
import cartopy.crs as ccrs
from pyresample.geometry import AreaDefinition
from pyresample import image, geometry, load_area, save_quicklook, SwathDefinition
from pyresample.kd_tree import resample_nearest
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile



## read in netcdf access data

In [ ]:
filename = 'F:/data/cruise_data/access/rockfish_casts_2011.nc'
ds = xr.open_dataset(filename)
ds = ds.swap_dims({'row':'time'})
ds = ds.rename({'latitude':'lat','longitude':'lon'})
ds_access1 = ds.copy(deep=True)
filename = 'F:/data/cruise_data/access/rockfish_casts_2015.nc'
ds = xr.open_dataset(filename)
ds = ds.swap_dims({'row':'time'})
ds = ds.rename({'latitude':'lat','longitude':'lon'})
ds_access2 = ds.copy(deep=True)
ds_access = xr.concat((ds_access1,ds_access2),dim='time')

## read in excel file
    

In [4]:
filename = 'F:/data/cruise_data/access/CTD_casts_ALL_NASA_update_010819.xlsx'
df = pd.read_excel(filename, sheetname='data')
df

ID_Date   Cruise       Date     Line Station CTD_Cast  \
0       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
1       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
2       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
3       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
4       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
5       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
6       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
7       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
8       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
9       20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
10      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
11      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
12      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
13      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
14      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
15      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
16      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
17      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
18      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
19      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
20      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
21      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
22      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
23      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
24      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
25      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
26      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
27      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
28      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
29      20100518-CB3_P09-C-10a  ACC1005 2010-05-18  CB3_P09       C      10a   
...                        ...      ...        ...      ...     ...      ...   
146310         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146311         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146312         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146313         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146314         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146315         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146316         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146317         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146318         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146319         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146320         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146321         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146322         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146323         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146324         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146325         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146326         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146327         20180928-4-E-31  ACC1809 2018-09-28        4       E       31   
146328         20180928

In [ ]:
#xarray.Dataset.from_dataframe(df.set_index(['X', 'Y', 'Date'])